<a href="https://colab.research.google.com/github/keduog/LLM/blob/main/llmswithoutkg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers datasets stanza torch torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [5]:
!pip install conllu

In [10]:
!pip install --upgrade datasets

In [12]:
import datasets
print(datasets.__version__)

3.3.1


In [13]:
from datasets import get_dataset_config_names

# List configurations for the Universal Dependencies dataset
ud_configs = get_dataset_config_names("universal_dependencies")
print(ud_configs)

['af_afribooms', 'akk_pisandub', 'akk_riao', 'aqz_tudet', 'sq_tsa', 'am_att', 'grc_perseus', 'grc_proiel', 'apu_ufpa', 'ar_nyuad', 'ar_padt', 'ar_pud', 'hy_armtdp', 'aii_as', 'bm_crb', 'eu_bdt', 'be_hse', 'bho_bhtb', 'br_keb', 'bg_btb', 'bxr_bdt', 'yue_hk', 'ca_ancora', 'zh_cfl', 'zh_gsd', 'zh_gsdsimp', 'zh_hk', 'zh_pud', 'ckt_hse', 'lzh_kyoto', 'cop_scriptorium', 'hr_set', 'cs_cac', 'cs_cltt', 'cs_fictree', 'cs_pdt', 'cs_pud', 'da_ddt', 'nl_alpino', 'nl_lassysmall', 'en_esl', 'en_ewt', 'en_gum', 'en_gumreddit', 'en_lines', 'en_partut', 'en_pronouns', 'en_pud', 'myv_jr', 'et_edt', 'et_ewt', 'fo_farpahc', 'fo_oft', 'fi_ftb', 'fi_ood', 'fi_pud', 'fi_tdt', 'fr_fqb', 'fr_ftb', 'fr_gsd', 'fr_partut', 'fr_pud', 'fr_sequoia', 'fr_spoken', 'gl_ctg', 'gl_treegal', 'de_gsd', 'de_hdt', 'de_lit', 'de_pud', 'got_proiel', 'el_gdt', 'he_htb', 'qhe_hiencs', 'hi_hdtb', 'hi_pud', 'hu_szeged', 'is_icepahc', 'is_pud', 'id_csui', 'id_gsd', 'id_pud', 'ga_idt', 'it_isdt', 'it_partut', 'it_postwita', 'it_pud'

In [14]:
from datasets import load_dataset

# Load the UD Amharic dataset
dataset = load_dataset("universal_dependencies", "am_att")

# Inspect the dataset
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 1074
    })
})


In [15]:
# Split the test data into train and validation sets
train_test_split = dataset["test"].train_test_split(test_size=0.2)
train_data = train_test_split["train"]
dev_data = train_test_split["test"]  # Use this as the validation set

In [17]:
# Extract all unique dependency labels from the dataset
dependency_labels = set()
for example in train_data:
    dependency_labels.update(example["deprel"])

num_dependency_labels = len(dependency_labels)
print(f"Number of dependency labels: {num_dependency_labels}")

Number of dependency labels: 35


In [18]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_dependency_labels)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments

# Load the UD Amharic dataset
dataset = load_dataset("universal_dependencies", "am_att")

# Split the test data into train and validation sets
train_test_split = dataset["test"].train_test_split(test_size=0.2)
train_data = train_test_split["train"]
dev_data = train_test_split["test"]  # Use this as the validation set

# Extract all unique dependency labels
dependency_labels = set()
for example in train_data:
    dependency_labels.update(example["deprel"])

num_dependency_labels = len(dependency_labels)
print(f"Number of dependency labels: {num_dependency_labels}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_dependency_labels)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["tokens"], truncation=True, padding="max_length", is_split_into_words=True)

train_dataset = train_data.map(tokenize_function, batched=True)
dev_dataset = dev_data.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

# Fine-tune the model
trainer.train()

Number of dependency labels: 34


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/859 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kediryassin25 (kediryassin25-gondar-university-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [20]:
# Extract all unique dependency labels
dependency_labels = set()
for example in train_data:
    dependency_labels.update(example["deprel"])

# Create a label-to-id mapping
label_to_id = {label: idx for idx, label in enumerate(sorted(dependency_labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}

num_dependency_labels = len(label_to_id)
print(f"Number of dependency labels: {num_dependency_labels}")
print(f"Label to ID mapping: {label_to_id}")

Number of dependency labels: 34
Label to ID mapping: {'_': 0, 'acl': 1, 'advcl': 2, 'advmod': 3, 'amod': 4, 'aux': 5, 'case': 6, 'cc': 7, 'ccomp': 8, 'compound': 9, 'compound:svc': 10, 'conj': 11, 'cop': 12, 'csubj': 13, 'csubj:pass': 14, 'dep': 15, 'det': 16, 'discourse': 17, 'expl': 18, 'fixed': 19, 'flat': 20, 'goeswith': 21, 'iobj': 22, 'mark': 23, 'nmod': 24, 'nsubj': 25, 'nsubj:pass': 26, 'nummod': 27, 'obj': 28, 'obl': 29, 'parataxis': 30, 'punct': 31, 'root': 32, 'xcomp': 33}


In [21]:
def tokenize_function(examples):
    # Tokenize the input
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding="max_length", is_split_into_words=True)

    # Map dependency labels to IDs
    labels = []
    for i, deprel in enumerate(examples["deprel"]):
        label_ids = [label_to_id[label] for label in deprel]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_dependency_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
def tokenize_function(examples):
    # Tokenize the input
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True,
    )

    # Map dependency labels to IDs
    labels = []
    for i, deprel in enumerate(examples["deprel"]):
        # Get the word IDs for the tokenized input
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens (e.g., [CLS], [SEP]) have word_idx = None
            if word_idx is None:
                label_ids.append(-100)  # Ignore these tokens in the loss computation
            # Only label the first token of each word
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[deprel[word_idx]])
            else:
                label_ids.append(-100)  # Ignore subsequent subword tokens
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [25]:
train_dataset = train_data.map(tokenize_function, batched=True)
dev_dataset = dev_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/859 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

In [26]:
print(train_dataset[0]["input_ids"])  # Tokenized input
print(train_dataset[0]["labels"])    # Corresponding labels

[101, 100, 100, 100, 100, 100, 100, 100, 100, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [27]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import wandb

# Log in to Weights & Biases
wandb.login()

# Load the UD Amharic dataset
dataset = load_dataset("universal_dependencies", "am_att")

# Split the test data into train and validation sets
train_test_split = dataset["test"].train_test_split(test_size=0.2)
train_data = train_test_split["train"]
dev_data = train_test_split["test"]  # Use this as the validation set

# Extract all unique dependency labels
dependency_labels = set()
for example in train_data:
    dependency_labels.update(example["deprel"])

# Create a label-to-id mapping
label_to_id = {label: idx for idx, label in enumerate(sorted(dependency_labels))}
id_to_label = {idx: label for label, idx in label_to_id.items()}

num_dependency_labels = len(label_to_id)
print(f"Number of dependency labels: {num_dependency_labels}")
print(f"Label to ID mapping: {label_to_id}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModelForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=num_dependency_labels)

# Tokenize the dataset
def tokenize_function(examples):
    # Tokenize the input
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True,
    )

    # Map dependency labels to IDs
    labels = []
    for i, deprel in enumerate(examples["deprel"]):
        # Get the word IDs for the tokenized input
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens (e.g., [CLS], [SEP]) have word_idx = None
            if word_idx is None:
                label_ids.append(-100)  # Ignore these tokens in the loss computation
            # Only label the first token of each word
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[deprel[word_idx]])
            else:
                label_ids.append(-100)  # Ignore subsequent subword tokens
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

train_dataset = train_data.map(tokenize_function, batched=True)
dev_dataset = dev_data.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="amharic-dependency-parsing",  # Unique run name
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
)

# Fine-tune the model
trainer.train()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Number of dependency labels: 35
Label to ID mapping: {'_': 0, 'acl': 1, 'advcl': 2, 'advmod': 3, 'amod': 4, 'aux': 5, 'case': 6, 'cc': 7, 'ccomp': 8, 'clf': 9, 'compound': 10, 'compound:svc': 11, 'conj': 12, 'cop': 13, 'csubj': 14, 'csubj:pass': 15, 'dep': 16, 'det': 17, 'discourse': 18, 'expl': 19, 'fixed': 20, 'flat': 21, 'goeswith': 22, 'iobj': 23, 'mark': 24, 'nmod': 25, 'nsubj': 26, 'nsubj:pass': 27, 'nummod': 28, 'obj': 29, 'obl': 30, 'parataxis': 31, 'punct': 32, 'root': 33, 'xcomp': 34}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/859 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.267051
2,No log,2.134986
3,No log,2.114447


TrainOutput(global_step=162, training_loss=2.3068128044222607, metrics={'train_runtime': 293.1838, 'train_samples_per_second': 8.79, 'train_steps_per_second': 0.553, 'total_flos': 673562640245760.0, 'train_loss': 2.3068128044222607, 'epoch': 3.0})

In [28]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [30]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=77a13ba30527b0b69202c339efe26bdb00be868d0c8dda83fd33e19ccb207c97
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [32]:
import evaluate

# Load the seqeval metric
seqeval = evaluate.load("seqeval")

In [33]:
import numpy as np
import evaluate

# Load the seqeval metric
seqeval = evaluate.load("seqeval")

# Function to compute metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored labels (where label = -100)
    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute UAS and LAS
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "UAS": results["overall_accuracy"],
        "LAS": results["overall_f1"],
    }

# Update the Trainer with the compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics,  # Add this line
)

# Evaluate the model
results = trainer.evaluate()
print(results)

{'eval_loss': 2.1144473552703857, 'eval_model_preparation_time': 0.0027, 'eval_UAS': 0.3715415019762846, 'eval_LAS': 0.2206546275395034, 'eval_runtime': 5.972, 'eval_samples_per_second': 36.001, 'eval_steps_per_second': 4.521}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: _ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: obj seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: det seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: case seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: compound:svc seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py: